# Funções para gerar o relatório das linhas de ônibus.

## Funções para tratamento dos dados da operação.

In [ ]:
import pandas as pd 
import numpy as np
import analises_config as anc
import glob
from pandas.core.common import flatten

In [ ]:
def abre_csv(arquivo_csv):
    '''Renomeia colunas e remove dados duplicados.

    :param arquivo_csv: arquivo csv com os dados das viagens de ônibus.
    :return: Dataframe com os dados da operação.
    '''
    df = pd.read_csv(arquivo_csv)
    df = df.rename(columns={'1':'Veiculo Ida', '2':'Sai Term Prev', '3':'Sai Term Real', '4':'Cheg Pt Ret Prev',
                            '5':'Cheg Pt Ret Real', '6': 'Tempo Ida Prev', '7':'Tempo Ida Real', '8': 'Veiculo Vol',
                            '9':'Saida Pt Ret Prev', '10':'Saida Pt Ret Real', '11':'Cheg Term Prev', 
                            '12':'Cheg Term Real', '13':'Tempo Vol Prev', '14':'Tempo Vol Real'})

    df.drop_duplicates(subset=None, keep="first", inplace=True)
    
    return df

In [ ]:
def conta_viagem(linha):
    '''Analisa as linhas de dados das viagens e informa a quantidade de viagens realizadas (0, 0,5 ou 1), ou seja, 0 viagem, meia viagem ou viagem completa.

    :param linha: Linhas do dataframe com os dados das viagens.
    :return: Retorna 0, 0,5 ou 1.
    '''
    if pd.isnull(linha['Tempo Ida Real']) and pd.isnull(linha['Tempo Vol Real']):
        valor = 0
    elif pd.notnull(linha['Tempo Ida Real']) and pd.isnull(linha['Tempo Vol Real']):
        valor = 0.5
    elif pd.isnull(linha['Tempo Ida Real']) and pd.notnull(linha['Tempo Vol Real']):
        valor = 0.5
    else:
        valor = 1
    return valor

In [ ]:
def cria_relatorio_veiculos(df):
    '''Conta a quantidade de veículos por linha, empresa e data. O que não for identificação de veículo na coluna 'Veiculo ida' é eliminado.
    
    :param df: Dataframe com os dados das viagens.
    :return: Retorna a contagem de veículos por linha, empresa e data.
    '''
    df['Veiculo Ida'] = pd.to_numeric(df['Veiculo Ida'], errors='coerce')
    df = df[df['Veiculo Ida'].notna()]    
    relatorio_veic = df.groupby(['Data','Empresa','Linha'])['Veiculo Ida'].nunique().reset_index(name='Qtd veículos')

    return relatorio_veic

In [ ]:
def cria_relatorio_viagens(df):
    '''Conta as viagens.
    
    :param df: Dataframe com os dados das viagens.
    :return: Retorna a contagem de viagens. 
    '''
    relatorio_viag = df.groupby(['Data','Empresa','Linha'])['contador_viagens'].sum().reset_index(name='Qtd viagens')

    return relatorio_viag

In [ ]:
def renomeia_linhas(df):
    '''Extrai o código da linha da coluna "Linha".

    :param df: Recebe o dataframe.
    :return: Retorna o dataframe com a extração do código da linha da coluna "Linha", reescrevendo a mesma.
    '''
    df['Linha'] = df['Linha'].str[0:4]
    df['Linha'] = df['Linha'].str.strip()

    return df

In [ ]:
def renomeia_empresas(df):
    '''Substitui os dados da coluna "Empresa" de acordo com o dicionário.

    :param df: Recebe o dataframe.
    :return: Retorna o dataframe com os códigos das empresas na coluna "Empresa" substituídos de acordo com o dicionário.
    '''
    df['Empresa'].replace(anc.dicionario_de_nomes_empresas, inplace=True)

    return df

In [ ]:
def substitui_nomes_de_linhas_errados(df):
    '''Substitui os códigos das linhas que estão errados na coluna "Linha".

    :param df: Recebe o dataframe da operação.
    :return: Retorna o dataframe com os códigos da coluna "Linha", que estão errados, substituídos
    '''
    df['Linha'].replace(anc.dicionario_de_linhas_erradas, inplace=True)

    return df

In [ ]:
def remove_zeros_a_esquerda(relatorio):
    '''Remove os zeros a esquerda.

    :param relatorio: Dataframe com o relatório da operação.
    :return: Retorna o dataframe com os zeros a esquerda da coluna "Linha", removidos.
    '''

    relatorio['Linha'] = relatorio['Linha'].apply(lambda x: x.lstrip('0'))

    return relatorio

In [ ]:
def remove_colunas(operacao):
    '''Remove as colunas que não serão utilizadas (Vindas dos arquivos txt).

    :param operacao: Dataframe com os dados da operação.
    :return: Retorna o mesmo dataframe mas sem as colunas que não serão utilizadas.
    '''

    operacao = operacao.drop(relatorio_operacao.columns[[3, 4, 5, 6, 7, 10, 11, 12, 13, 14]], axis=1) 

    return operacao

In [ ]:
def remove_linhas_sem_acesso(operacao):
    '''Remove as linhas de ônibus que não temos acesso.

    :param operacao: Dataframe com o relatório da operação.
    :return: Retorna o mesmo dataframe sem as linhas que não temos acesso.
    '''
    operacao = operacao[~operacao['Empresa'].isin(anc.sem_acesso)]

    return operacao

## Funções para tratamento dos dados da programação.

In [ ]:
def limpa_arquivo_programacao(df):
    '''Faz o tratamento dos dados da programação.

    :param df: Dataframe com o arquivo da programação.
    :return: Retorna o dataframe sem os valores nulos e outros valores da coluna Empresa e data, bem como sem os pontos nos códigos das linhas.
    '''
    df = df[df['Empresa'] != 'Empresa']
    df = df[df['Empresa'] != 'GERAL']
    df = df[df['Data'].notna()]
    df['Linha'] = df['Linha'].astype(str).str.split('.').str[0]
    
    return df

In [ ]:
def cria_relatorio_da_operacao(programacao, relatorio):
    '''Funde os dataframes da programação e da operação.

    :param programacao: Dataframe com os dados da programação.
    :param relatorio: Dataframe com os dados do relatório da operação.
    :return: Retorna um dataframe resultado da fusão dos dois anteriores.
    '''
    programacao['Linha'] = programacao['Linha'].astype(str)
    relatorio_op = pd.merge(programacao, relatorio,  how='left', left_on=['Data','Empresa', 'Linha'], right_on = ['Data','Empresa', 'Linha'])

    return relatorio_op

In [ ]:
def ordena_colunas(relatorio):
    '''Ordena as colunas.

    :param relatorio: Dataframe com relatório resultado da fusão da programação com operação.
    :return: Retorna o mesmo dataframe com as colunas ordenadas.
    '''
    relatorio_organizado = relatorio[['Data', 'Empresa', 'Linha', 'Viagens Programadas', 'Viagens realizadas', 'Porcentagem de viagens', 
                                    'Veiculos Programados', 'Veiculos em operacao', 'Porcentagem de veiculos']]
    return relatorio_organizado

In [ ]:
def reduz_frota_operando_ao_programado(relatorio):
    '''Verifica se a frota em operação é maior que a programada e caso positivo, altera o valor deixando igual ao programado.

    :param relatorio: Dataframe com relatório resultado da fusão da programação com operação.
    :return: Retorna o dataframe com os valores de frota em operação reduzidos conforme programação.
    '''
    relatorio['Veiculos em operacao'].loc[(relatorio['Veiculos em operacao'].astype('int') > relatorio['Veiculos Programados'].astype('int'))] = \
        relatorio['Veiculos Programados']

    return relatorio

In [ ]:
def reduz_viagens_realizadas_ao_programado(relatorio):
    '''Verifica se as viagens realizadas são maiores que as programadas e caso positivo, altera o valor deixando igual ao
    programado.

    :param relatorio: Dataframe com relatório resultado da fusão da programação com operação.
    :return: Retorna o dataframe com os valores de viagens realizadas reduzidos conforme programação.
    '''
    relatorio['Viagens realizadas'].loc[(relatorio['Viagens realizadas'].astype('float') > relatorio['Viagens Programadas'].astype('float'))] = \
        relatorio['Viagens Programadas']

    return relatorio

In [ ]:
def abre_arquivo_das_programacoes():
    '''Carrega os dados de programação dos arquivos txt.

    :return: Dataframe com os dados dos arquivos de programação.
    '''
    l = [pd.read_csv(filename) for filename in glob.glob(anc.pasta_dados_de_programacao)]
    programacao = pd.concat(l, axis=0)
    programacao = limpa_arquivo_programacao(programacao)

    return programacao

In [ ]:
def renomeia_colunas_do_relatorio(operacao):
    '''Renomeia as colunas do dataframe do relatório.

    :param relatorio: Dataframe com o relatório da operação.
    :return: Retorna o mesmo dataframe com as colunas renomeadas.
    '''
    operacao = operacao.rename(columns={'Viag.':'Viagens Programadas', 'Veíc.':'Veiculos Programados', 
                                                'Qtd veículos':'Veiculos em operacao', 'Qtd viagens': 'Viagens realizadas'})

    return operacao

In [ ]:
def substitui_virgula_por_ponto(operacao):
    '''Substitui as vírgulas por ponto da coluna Viagens Programadas.

    :param operacao: Dataframe com o relatório da operação.
    :return: Retorna o mesmo dataframe com as substituições.
    '''
    operacao['Viagens Programadas'] = operacao['Viagens Programadas'].apply(lambda x: x.replace(',','.'))

    return operacao

In [ ]:
def substitui_nulos_por_zero(operacao, coluna):
    '''Substitui os valores NAN da coluna recebida por parâmetro, pelo valor 0.

    :param operacao: Dataframe com o relatório.
    :param coluna: Nome da coluna do dataframe.
    :return: Retorna o mesmo dataframe com os valores da coluna que forem NAN substituídos por 0.
    '''
    operacao[coluna] = operacao[coluna].fillna(0)

    return operacao

#Funções para gerar os relatórios consolidados por empresa.

In [ ]:
def cria_coluna_com_veiculos_efetivos(df):
    '''Remove os dados da coluna "Veiculo ida" que não são códigos dos ônibus (SCOM, VNRC, etc)

    :param df: Dataframe com os dados das viagens.
    :return: retorna o mesmo dataframe contendo somente os veículos da coluna "Veiculo ida"(Remove SCOM, VNRC ...).
    '''
    df['Veiculo Ida'] = pd.to_numeric(df['Veiculo Ida'], errors='coerce').astype(pd.Int64Dtype())

    return df

In [ ]:
def cria_dataframe_com_listas_de_veiculos(df_gps):
    '''Cria um dataframe incluindo a lista de veículos que operaram(Com repetição).

    :param df_gps: Dataframe com os dados das viagens.
    :return: Retorna um dataframe incluindo a lista de veículos que operaram(Com repetição).
    '''
    dados_com_lista = df_gps.groupby(['Data', 'Empresa', 'Linha'])['Veiculo Ida'].apply(list).reset_index(name='Listas veiculos')

    return dados_com_lista

In [ ]:
def remove_veiculos_repetidos(df_relatorio):
    '''Remove veículos repetidos na lista.

    :param df_relatorio: Dataframe com os dados incluindo as listas de veículos.
    :return: Retorna o mesmo dataframe, onde os veículos repetidos na lista foram removidos.
    '''
    for indice, linha in df_relatorio.iterrows():
        lista_atual = df_relatorio.loc[indice, 'Listas veiculos']
        lista = [valor for valor in lista_atual if str(valor).isnumeric()]
        lista = list(dict.fromkeys(lista))
        df_relatorio.loc[indice, 'Listas veiculos'] = lista

    return df_relatorio

In [ ]:
def remove_veiculos_alem_do_programado(df_relatorio):
    '''Elimina veículos das listas, se a quantidade de veículos for maior que a quantidade programada.

    :param df_relatorio: Dataframe com os dados da operação já com a lista de veículos únicos.
    :return: Retorna a mesma lista com os N veículos programados por linha de acordo com a programação(Eliminando veículos 
    a mais que o programado como exemplo as substituíções).
    '''
    df_relatorio['N primeiros veic prog'] = np.empty((len(df_relatorio), 0)).tolist()

    for indice, linha in df_relatorio.iterrows():
        lista_atual = df_relatorio.loc[indice, 'Listas veiculos']
        prog_veic_linha_atual = df_relatorio.loc[indice, 'Veíc.']
        if isinstance(lista_atual, list):
            lista_primeiros_n_programados = lista_atual[:int(prog_veic_linha_atual)]
            df_relatorio.at[indice, 'N primeiros veic prog'] = lista_primeiros_n_programados
        else:
            df_relatorio.loc[indice, 'N primeiros veic prog'] = float('nan') 

    return df_relatorio

In [ ]:
def agrupa_empresas(df_relatorio):
    '''Cria lista de listas dos veículos que operaram nas linhas, agrupadas por empresa.

    :param df_relatorio: Dataframe com as listas de veículos nas linhas.
    :return: Retorna um dataframe com lista de listas dos veículos que operaram nas linhas, agrupadas por empresa.
    '''
    df_agrupado = df_relatorio.groupby(['Data', 'Empresa'])['N primeiros veic prog'].apply(list).reset_index(name='Listas veiculos empresa')

    return df_agrupado

In [ ]:
def cria_relatorio_final_de_veiculos(df_relatorio):
    '''Cria dataframe com a quantidade de veículos utilizados pelas empresas por data.

    :param df_relatorio: Dataframe com as listas de listas dos veículos que operaram nas linhas.
    :return: Retorna um dataframe com a quantidade de veículos utilizados pelas empresas por data.
    '''
    df_relatorio['Listas veiculos empresa unicos'] = np.empty((len(df_relatorio), 0)).tolist()
    df_relatorio['Veiculos operando'] = '0'

    lista_final = []
    for indice, linha in df_relatorio.iterrows():
            lista_atual = df_relatorio.loc[indice, 'Listas veiculos empresa']
            lista_final = list(flatten(lista_atual))
            lista_final = [valor for valor in lista_final if str(valor).isnumeric()]
            lista_final = list(dict.fromkeys(lista_final))
            df_relatorio.loc[indice, 'Listas veiculos empresa unicos'] = lista_final
            df_relatorio.loc[indice, 'Veiculos operando'] = len(lista_final)

    return df_relatorio

In [ ]:
def soma_qtd_viagens_por_empresa(df_relatorio_linhas):
    '''Cria o dataframe com a quantidade de viagens realizadas por empresa.

    :param df_relatorio_linhas: Dataframe com os dados do arquivo resultado da análise por linhas.
    :return: Retorna um dataframe com a quantidade de viagens realizadas por empresa.
    '''
    df_soma_viagens = df_relatorio_linhas.groupby(['Data', 'Empresa'])['Viagens realizadas'].agg('sum').reset_index(name='Viagens Realizadas')

    return df_soma_viagens

In [ ]:
def abre_arquivo_dados_por_linha(arquivo_excel):
    '''Abre o arquivo excel gerado pela análise por linhas.

    :param arquivo_excel: Caminho do arquivo excel com os dados resultado da análise por linhas.
    :return: Retorna um dataframe com os dados.
    '''
    df_saida = pd.read_excel(arquivo_excel)

    return df_saida

In [ ]:
def funde_frota_e_viagens(df_frota, df_viagens):
    '''Funde os dataframes com os dados da frota em operação e o de viagens realizadas por empresa e data.

    :param df_frota: Dataframe com os dados de frota em operação.
    :param df_viagens: Dataframe com os dados de viagens.
    :return: Retorna os dataframes fundidos.
    '''
    df_total = pd.merge(df_frota, df_viagens,  how='left', left_on=['Data','Empresa'], right_on = ['Data','Empresa'])

    return df_total

In [ ]:
def cria_relatorio_final(df_viag_frot, df_prog):
    '''Cria relatório final com viagens e frotas programadas e realizadas por empresa.

    :param df_viag_frot: Dataframe com viagens e frota realizadas.
    :param df_prog: Dataframe com programação.
    :return: Retorna o relatório final com viagens e frotas programadas e realizadas por empresa.
    '''

    df_prog['Viag.'] = df_prog['Viag.'].astype(float)
    df_prog['Veíc.'] = df_prog['Veíc.'].astype(int)

    df_viag_prog_por_emp = df_prog.groupby(['Data', 'Empresa'])['Viag.'].agg('sum').reset_index(name='Viagens programadas')
    df_frot_prog_por_emp = df_prog.groupby(['Data', 'Empresa'])['Veíc.'].agg('sum').reset_index(name='Frota programada')

    df_saida = pd.merge(df_viag_prog_por_emp, df_viag_frot,  how='left', left_on=['Data','Empresa'], right_on = ['Data','Empresa'])
    df_saida = pd.merge(df_saida, df_frot_prog_por_emp,  how='left', left_on=['Data','Empresa'], right_on = ['Data','Empresa'])

    return df_saida

In [ ]:
def organiza_relatorio_final(df_relatorio):
    '''Organiza as colunas do dataframe.

    :param df_relatorio: Dataframe com o relatório final das empresas.
    :return: Retorna o dataframe com a ordem das colunas organizadas.
    '''
    df_relatorio = df_relatorio[['Data', 'Empresa', 'Viagens programadas', 'Viagens Realizadas', 'Porcentagem de viagens', 'Frota programada',
                                 'Veiculos operando', 'Porcentagem de veiculos']]
    df_relatorio = df_relatorio.sort_values(['Data', 'Empresa'], ascending=[True, True])

    return df_relatorio

#Funções que realizam as análises por faixas horárias.

In [ ]:
import pandas as pd
import faixas_config as conf
import glob
from datetime import datetime
from datetime import timedelta

In [ ]:
def abre_arquivos_xlsx(pasta_dos_arquivos):
    '''Cria um dataframe com os dados de arquivos XLSX.

    :param pasta_dos_arquivos: Pasta que contêm os arquivos XLSXs com os dados.
    :return: Retorna um dataframe com os arquivos XLSXs concatenados.
    '''
    all_files = glob.glob(pasta_dos_arquivos + "/*.xlsx")
    li = []
    for filename in all_files:
        df = pd.read_excel(filename, index_col=None, header=0)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)

    return frame

In [ ]:
def conta_viagem_programada(linha):
    '''Conta as viagens programadas.

    :param linha: linhas com os dados das viagens.
    :return: Retorna o valor informando 0 viagem, meia viagem ou viagem completa(0, 0.5, 1).
    '''
    if pd.isnull(linha['SAIDA TERMINAL']) and pd.notnull(linha['CHEGADA TERMINAL']):
        valor = 0.5
    elif pd.notnull(linha['SAIDA TERMINAL']) and pd.isnull(linha['CHEGADA TERMINAL']):
        valor = 0.5
    else:
        valor = 1
    return valor

In [ ]:
def trata_saida_terminal_em_branco(df):
    '''Testa se o dado da coluna SAIDA TERMINAL está vazio, caso positivo a função iguala ao valor da coluna SAIDA RETORNO.

    :param df: Dataframe com as programações.
    :return: O dataframe de entrada com a coluna SAIDA TERMINAL corrigida.
    '''
    df['SAIDA TERMINAL'].fillna(df['SAIDA RETORNO'], inplace=True)

    return df

In [ ]:
def trata_chegada_terminal_em_branco(df):
    '''Testa se o dado da coluna CHEGADA TERMINAL está vazio, caso positivo a função iguala ao valor da coluna CHEGADA RETORNO.

    :param df: Dataframe com as programações.
    :return: O dataframe de entrada com a coluna CHEGADA TERMINAL corrigida.
    '''
    df['CHEGADA TERMINAL'].fillna(df['CHEGADA RETORNO'], inplace=True)

    return df 

In [ ]:
def trata_virada_dia(df, opcao):
    '''Trata a virada de dia (Hora igual ou acima de 24).
    

    :param df: Dataframe da programação já com as colunas de data e hora concatenadas.
    :opcao: opção de coluna: st se refere a saída de terminal, cpr a chegada no ponto de retorno, spr a saída do ponto de retorno e ct a chegada no terminal.
    :return: Retorna o dataframe já tratado.
    '''

    if opcao == 'st':
        coluna = 'ST'
    elif opcao == 'cpr':
        coluna = 'CPR'
    elif opcao == 'spr':
        coluna = 'SPR'
    elif opcao == 'ct':
        coluna = 'CT'

    for index, row in df.iterrows():
        if str(row[coluna])[11:13] == '24':
            hora_temp ='00'
            data_temp = datetime.strptime(row[coluna][0:10], '%Y-%m-%d')
            data_temp = data_temp + timedelta(days=1)
            df.at[index,coluna] = str(data_temp)[:10] + ' ' + hora_temp + row[coluna][13:]
        elif str(row[coluna])[11:13] == '25':
            hora_temp ='01'
            data_temp = datetime.strptime(row[coluna][0:10], '%Y-%m-%d')
            data_temp = data_temp + timedelta(days=1)
            df.at[index,coluna] = str(data_temp)[:10] + ' ' + hora_temp + row[coluna][13:]
        elif str(row[coluna])[11:13] == '26':
            hora_temp ='02'
            data_temp = datetime.strptime(row[coluna][0:10], '%Y-%m-%d')
            data_temp = data_temp + timedelta(days=1)
            df.at[index,coluna] = str(data_temp)[:10] + ' ' + hora_temp + row[coluna][13:]            
        elif str(row[coluna])[11:13] == '27':
            hora_temp ='03'
            data_temp = datetime.strptime(row[coluna][0:10], '%Y-%m-%d')
            data_temp = data_temp + timedelta(days=1)
            df.at[index,coluna] = str(data_temp)[:10] + ' ' + hora_temp + row[coluna][13:]
        elif str(row[coluna])[11:13] == '28':
            hora_temp ='04'
            data_temp = datetime.strptime(row[coluna][0:10], '%Y-%m-%d')
            data_temp = data_temp + timedelta(days=1)
            df.at[index,coluna] = str(data_temp)[:10] + ' ' + hora_temp + row[coluna][13:]
        elif str(row[coluna])[11:13] == '29':
            hora_temp ='05'
            data_temp = datetime.strptime(row[coluna][0:10], '%Y-%m-%d')
            data_temp = data_temp + timedelta(days=1)
            df.at[index,coluna] = str(data_temp)[:10] + ' ' + hora_temp + row[coluna][13:]
        elif str(row[coluna])[11:13] == '30':
            hora_temp ='06'
            data_temp = datetime.strptime(row[coluna][0:10], '%Y-%m-%d')
            data_temp = data_temp + timedelta(days=1)
            df.at[index,coluna] = str(data_temp)[:10] + ' ' + hora_temp + row[coluna][13:]

    return df

In [ ]:
def cria_coluna_horarios(df, opcao):
    '''Cria uma nova coluna formatada em datetime.

    :param df: Dataframe com os dados da programação.
    :param opcao: String com a opção de coluna. st se refere a saída de terminal, cpr a chegada no ponto de retorno,
     spr a saída do ponto de retorno e ct a chegada no terminal.
     :return: Retorna o dataframe com uma nova coluna formatada em datetime.
    '''
    if opcao == 'st':
        df['ST'] = df['data'] + ' ' + df['SAIDA TERMINAL']
        df = trata_virada_dia(df, opcao)
        df['ST'] =  pd.to_datetime(df['ST'], format='%Y-%m-%d %H:%M')

    elif opcao == 'cpr':
        df['CPR'] = df['data'] + ' ' + df['CHEGADA RETORNO']
        df = trata_virada_dia(df, opcao)
        df['CPR'] =  pd.to_datetime(df['CPR'], format='%Y-%m-%d %H:%M')

    elif opcao == 'spr':
        df['SPR'] = df['data'] + ' ' + df['SAIDA RETORNO']
        df = trata_virada_dia(df, opcao)
        df['SPR'] =  pd.to_datetime(df['SPR'], format='%Y-%m-%d %H:%M') 

    elif opcao == 'ct':
        df['CT'] = df['data'] + ' ' + df['CHEGADA TERMINAL']
        df = trata_virada_dia(df, opcao)
        df['CT'] =  pd.to_datetime(df['CT'], format='%Y-%m-%d %H:%M')      

    return df

In [ ]:
def cria_coluna_tempo_viagem_qh(df):
    '''Calcula o tempo de viagem programado.

    :param df: Dataframe da programação já com as colunas dos horários formatadas.
    :return: Retorna o tempo de viagem programado em uma nova coluna.
    '''
    df['TEMPO DE VIAGEM'] = df['CT'] - df['ST']
    for index, row in df.iterrows():
        df.at[index,'TEMPO DE VIAGEM'] = (row['TEMPO DE VIAGEM'].seconds//60)

    return df

In [ ]:
def cria_faixa_horaria_programacao(df):
    '''Calcula as viagens programadas por faixas horárias.

    :param df: Dataframe da programação.
    :return: Retorna o mesmo com uma coluna com as viagens programadas por faixas horárias.
    ''' 
    df['faixa_horaria'] = pd.to_datetime(df['ST']).dt.strftime('%H')
    
    return df

In [ ]:
def cria_qtd_viagens_por_faixa(df):
    '''Calcula a quantidade de viagens programadas por faixa horária.

    :param df: Dataframe da programação.
    :return: Retorna um novo com a quantidade de viagens programadas por faixa horária.
    '''
    df_qtd = df.groupby(['data', 'Empresa', 'Linha', 'faixa_horaria'])\
    ['contador_viagens'].sum().reset_index(name='Viagens programadas')
    
    return df_qtd 

In [ ]:
def troca_formato_da_date(df):
    '''Converte o formato de data.

    :param df: Dataframe.
    :return: Dataframe com uma nova coluna de data no formatada de yyyy-mm-dd para dd/mm/yyyy.
    '''
    df['data_formatada'] = pd.to_datetime(df['data'], format='%Y/%m/%d')
    df['data_formatada'] = df['data_formatada'].dt.strftime('%d/%m/%Y')
    
    return df

In [ ]:
def cria_media_entre_faixas_horarias_qh(df):
    '''Calcula as médias das programações entre as faixas horárias.

    :param df: Dataframe da programação.
    :return: retorna um dataframe com as médias das programações entre as faixas horárias.
    '''
    df_medias = df.groupby(['data', 'Empresa', 'Linha', 'faixa_horaria'])['TEMPO DE VIAGEM'].mean().reset_index(name='Medias dos tempos programados')
    df_medias['Medias dos tempos programados'] = df_medias['Medias dos tempos programados'].round(2)

    return df_medias

In [ ]:
def cria_qtd_veiculos_por_faixa_qh(df):
    '''Calcula a quantidade de veículos programados por faixa horária.

    :param df: Dataframe da programação.
    :return: Retorna um novo dataframe com a quantidade de veículos programados por faixa horária.
    '''
    df_qtd = df.groupby(['data', 'Empresa', 'Linha', 'faixa_horaria'])\
    ['POSICAO'].nunique().reset_index(name='Veículos programados')
    
    return df_qtd  

In [ ]:
def cria_dataframe_final_da_programacao(df_frota, df_viagens, df_tempos):
    '''Cria um dataframe com as programações de veículos e viagens por faixa horária bem como os tempos médios de viagem por faixa.

    :param df_frota: Dataframe com a frota programada por faixa horária.
    :param df_viagens: Dataframe com as viagens programadas por faixa horária.
    :param df_tempos: Dataframe com os tempos de viagens programados por faixa hoŕaria.
    :return: Retorna um dataframe com as programações de veículos e viagens por faixa horária bem como os tempos médios de viagem por faixa.
    '''
    df_frota['Linha'] = df_frota['Linha'].astype('str')
    df_combinado = pd.merge(df_frota, df_viagens,  how='left', left_on=['data', 'Empresa', 'Linha', 'faixa_horaria'],
                        right_on = ['data', 'Empresa', 'Linha', 'faixa_horaria'])
    
    df_tempos['Linha'] = df_tempos['Linha'].astype('str')
    df_combinado = pd.merge(df_combinado, df_tempos,  how='left', left_on=['data', 'Empresa', 'Linha', 'faixa_horaria'],
                        right_on = ['data', 'Empresa', 'Linha', 'faixa_horaria'])
    
    return df_combinado

In [ ]:
def abre_arquivos_csv(pasta_dos_csvs):
    '''Cria um dataframe com os arquivos csvs concatenados.

    :param pasta_dos_csvs: Endereço da pasta que contêm os arquivos csvs com os dados.
    :return: Retorna um dataframe com os arquivos csvs concatenados.
    '''
    all_files = glob.glob(pasta_dos_csvs + "/*.csv")
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)

    return frame

In [ ]:
def cria_coluna_tempo_viagem_gps(df):
    '''Cria uma coluna contendo o tempo total de viagem.

    :param df: Dataframe com os dados das viagens.
    :return: Retorna o dataframe com uma coluna contendo o tempo total de viagem.
    '''
    df['TEMPO TOTAL DE VIAGEM'] = df['Tempo Ida Real'] + df['Tempo Vol Real']

    return df

In [ ]:
def cria_faixa_horaria_gps(df):
    '''Cria uma coluna com as faixas horárias das viagens realizadas.

    :param df: Dataframe da operação.
    :return: Retorna o mesmo com uma coluna com as faixas horárias das viagens realizadas.
    '''
    df_temp = df.copy()
    df_temp['Sai Term Real'].fillna(df_temp['Saida Pt Ret Real'], inplace=True)
    df['faixa_horaria'] = pd.to_datetime(df_temp['Sai Term Real'],errors='coerce').dt.strftime('%H')
    
    return df

In [ ]:
def cria_coluna_tempo_viagem_gps(df):
    '''Cria uma coluna contendo o tempo total de viagem.

    :param df: Dataframe com os dados das viagens.
    :return: Retorna com uma coluna contendo o tempo total de viagem.
    '''
    df['TEMPO TOTAL DE VIAGEM'] = df[["Tempo Ida Real", "Tempo Vol Real"]].sum(axis=1)

    return df

In [ ]:
def cria_qtd_viagens_por_faixa(df):
    '''Calcula a quantidade de viagens realizadas por faixa horária.

    :param df: Dataframe das viagens.
    :return: Retorna um novo com a quantidade de viagens realizadas por faixa horária.
    '''
    df_qtd = df.groupby(['Data', 'Empresa', 'Linha', 'faixa_horaria'])\
    ['contador_viagens'].sum().reset_index(name='Viagens realizadas')
    
    return df_qtd   

In [ ]:
def cria_media_entre_faixas_horarias_gps(df):
    '''Calcula as médias dos tempos de viagens entre as faixas horárias.

    :param df: Dataframe com os dados das viagens.
    :return: retorna um dataframe com as médias dos tempos de viagens entre as faixas horárias.
    '''
    df_medias = df.groupby(['Data', 'Empresa', 'Linha', 'faixa_horaria'])\
        ['TEMPO TOTAL DE VIAGEM'].mean().reset_index(name='Medias dos tempos de viagens realizadas')
    df_medias['Medias dos tempos de viagens realizadas'] = df_medias['Medias dos tempos de viagens realizadas'].round(2)
    
    return df_medias

In [ ]:
def cria_qtd_veiculos_por_faixa_gps(df):
    '''Calcula a quantidade de veículos por faixa horária.
    :param df: Dataframe com os dados das viagens.
    :return: Retorna um novo dataframe com a quantidade de veículos por faixa horária.
    '''
    df_temp = df.copy()
    df_temp['Veiculo Ida'].fillna(df_temp['Veiculo Vol'], inplace=True)
    df_temp['Veiculo Ida'] = pd.to_numeric(df_temp['Veiculo Ida'], errors='coerce')
    
    df_qtd = df_temp.groupby(['Data', 'Empresa', 'Linha', 'faixa_horaria'])\
    ['Veiculo Ida'].nunique().reset_index(name='Veículos em operação')
    
    return df_qtd 

In [ ]:
def cria_dataframe_final_da_operacao(df_frota_gps, df_viagens_gps, df_tempos_gps):
    '''Cria um dataframe com as quantidades de veículos e viagens da operação por faixa horária bem como os tempos médios de viagem por faixa.

    :param df_frota_gps: Dataframe com a frota realizada por faixa horária.
    :param df_viagens_gps: Dataframe com a quantidade de viagens realizadas por faixa horária.
    :param df_tempos_gps: Dataframe com os tempos de viagens por faixa horária.
    :return: retorna um dataframe com as quantidades de veículos e viagens da operação por faixa horária bem como os tempos médios de viagem por faixa.
    '''
    df_combinado = pd.merge(df_frota_gps, df_viagens_gps,  how='left', left_on=['Data', 'Empresa', 'Linha', 'faixa_horaria'],
                        right_on = ['Data', 'Empresa', 'Linha', 'faixa_horaria'])
    
    df_combinado = pd.merge(df_combinado, df_tempos_gps,  how='left', left_on=['Data', 'Empresa', 'Linha', 'faixa_horaria'],
                        right_on = ['Data', 'Empresa', 'Linha', 'faixa_horaria'])
    
    return df_combinado

In [ ]:
def junta_dados_programacao_e_operacao(df_programacao, df_operacao):
    '''Cria um dataframe resultado da fusão dos dados da programação com os da operação.

    :param df_programacao: Dataframe com os dados da programação.
    :param df_operacao: Dataframe com os dados da operação.
    :return: retorna um dataframe com a fusão dos dois anteriores.
    '''
    df_merge = pd.merge(df_programacao, df_operacao,  how='left', left_on=['data_formatada', 'Empresa', 'Linha', 'faixa_horaria'],
                        right_on = ['Data', 'Empresa', 'Linha', 'faixa_horaria'])
    
    return df_merge